
# Adaptación al sitio de pronósticos solares WRF-SMN mediante Machine Learning (MLP)

**Autores:** Dr. Germán Ariel Salazar, Lic. Rubén Darío Ledesma  
**Institución:** SMN / INENCO – CONICET  
**Propósito:** Documentar de forma reproducible el flujo completo de trabajo para:
- Descargar y visualizar datos WRF-SMN
- Extraer series temporales en un punto con mediciones
- Comparar con GHI medida en superficie
- Analizar el desempeño del modelo
- Implementar adaptación al sitio mediante MLP

Este notebook funciona como **plantilla técnica extensa**, preparada para auditoría,
transferencia y reutilización.



## Contenidos

1. Introducción general  
2. Descarga y organización de datos WRF-SMN  
3. Visualización de productos (imágenes/mapas)  
4. Recuperación de datos para un punto (lat, lon)  
5. Manejo de corridas y lead times  
6. Series temporales del modelo  
7. Datos medidos de GHI: lectura y control de calidad  
8. Integración horaria y merge modelo–medición  
9. Análisis exploratorio completo (EDA)  
10. Métricas de desempeño del WRF-SMN  
11. Adaptación al sitio mediante MLP  
12. Evaluación del modelo adaptado  
13. Síntesis final



## 1. Librerías y configuración general

Se utilizan librerías estándar del ecosistema científico Python.


In [ ]:

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

import pvlib
from pathlib import Path

plt.rcParams['figure.figsize'] = (11,5)
plt.rcParams['font.size'] = 11
sns.set_style("whitegrid")



## 2. Descarga y organización de los datos WRF-SMN

Los datos del modelo WRF-SMN se encuentran disponibles públicamente a través de
**Amazon Web Services (AWS – Open Data Program)**.

### 2.1. Estructura típica de los archivos

- Formato: **NetCDF (.nc)**
- Resolución espacial: ~4 km
- Frecuencia temporal: horaria
- Horizonte de pronóstico: 72 h
- Corridas: 00, 06, 12 y 18 UTC

Cada archivo contiene:
- Variables meteorológicas
- Coordenadas espaciales (lat, lon)
- Tiempo (leadtime y valid_time)



### 2.2. Ejemplo de descarga (documental)

> ⚠️ Este notebook **no ejecuta la descarga automáticamente**.
> Se documenta el procedimiento para trazabilidad.

Ejemplo usando `aws s3`:

```bash
aws s3 sync s3://smn-ar-wrf-dataset/WRF_4KM/2023/ ./WRF_2023/
```

O bien usando `wget` sobre URLs públicas.



## 3. Visualización de productos WRF-SMN (mapas / imágenes)

Antes de trabajar con series temporales, es recomendable inspeccionar
espacialmente las variables del modelo.


In [ ]:

# Apertura de un archivo NetCDF de ejemplo
# ds = xr.open_dataset("WRF_2023_01_01_00.nc")

# Visualización espacial de la GHI acumulada o instantánea
# ds['ACSWDNB'].isel(time=12).plot()
# plt.title("Radiación de onda corta entrante – leadtime 12 h")
# plt.show()

pass



## 4. Recuperación de información para un punto P (latitud, longitud)

Dado un punto con mediciones (estación), se extrae la **celda más cercana**
del modelo WRF-SMN.


In [ ]:

def extract_point(ds, lat_p, lon_p):
    """
    Extrae la serie temporal del punto más cercano a (lat_p, lon_p).
    """
    dist = (ds['lat'] - lat_p)**2 + (ds['lon'] - lon_p)**2
    iy, ix = np.unravel_index(dist.argmin(), dist.shape)
    return ds.isel(lat=iy, lon=ix)

# Ejemplo:
# site = extract_point(ds, lat_p=-34.60, lon_p=-58.48)
# df_site = site.to_dataframe().reset_index()



## 5. Manejo de corridas y lead times

Cada pronóstico se define por:
- `date`   → fecha de inicio
- `corrida` → hora de inicio (00, 06, 12, 18)
- `leadtime` → horas desde el inicio
- `valid_time` → tiempo físico real

El análisis se realiza siempre en función de **valid_time**,
que permite comparar con mediciones.


In [ ]:

# Ejemplo de agrupamiento por corrida
# df_site.groupby('corrida')['GHI_Wm2'].mean()

pass



## 6. Series temporales del modelo por corrida

Se pueden visualizar:
- Todas las corridas superpuestas
- Una corrida específica
- Comparación por leadtime


In [ ]:

# Ejemplo de ploteo de series por corrida
# for c in [0,6,12,18]:
#     df_site[df_site['corrida']==c].set_index('valid_time')['GHI_Wm2'].plot(label=f'corrida {c}')

# plt.legend()
# plt.ylabel("GHI [W/m²]")
# plt.title("Serie temporal WRF-SMN por corrida")
# plt.show()

pass



## 7. Datos medidos de GHI en superficie

Los datos medidos:
- Frecuencia: 1 minuto
- Se someten a control de calidad
- Luego se integran a frecuencia horaria


In [ ]:

# df_meas = pd.read_csv("GHI_medida_1min.csv", parse_dates=['time'])

# Cálculo del ángulo cenital
# df_meas['SZA'] = pvlib.solarposition.get_solarposition(
#     df_meas['time'], lat, lon)['apparent_zenith']

# Filtrado día / noche
# df_meas = df_meas[df_meas['SZA'] < 90]

pass



## 8. Integración horaria y merge con el modelo

Convención:
- La hora HH:00 representa el promedio de HH:00–HH:59


In [ ]:

# df_hour = (
#     df_meas
#     .set_index('time')
#     .resample('H')
#     .mean()
#     .dropna()
# )

# Merge con WRF-SMN usando valid_time
# df_merged = pd.merge(df_site, df_hour, left_on='valid_time', right_index=True)

pass



## 9. Análisis exploratorio completo (EDA)

Se analizan:
- Distribuciones
- Estadísticos descriptivos
- Correlaciones
- Relación con GHI medida


In [ ]:

# Estadísticos básicos
# df_merged.describe()

# Matriz de correlación
# corr = df_merged.corr()
# sns.heatmap(corr, cmap='coolwarm', center=0)
# plt.title("Matriz de correlación WRF-SMN + GHI medida")
# plt.show()

pass



## 10. Métricas de desempeño del WRF-SMN

Se utilizan:
- MBE
- RMSE
- rMBE
- rRMSE


In [ ]:

def metrics(y_true, y_pred):
    mbe = np.mean(y_pred - y_true)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r_mbe = 100 * mbe / np.mean(y_true)
    r_rmse = 100 * rmse / np.mean(y_true)
    return mbe, rmse, r_mbe, r_rmse

# metrics(df_merged['GHI_medida'], df_merged['GHI_Wm2'])

pass



## 11. Adaptación al sitio mediante MLP

Se implementa un **Multilayer Perceptron no secuencial**
para corregir la salida del WRF-SMN.


In [ ]:

features = ['PP','HR2','T2','PSFC','TSLB','SMOIS','GHI_Wm2']
target = 'GHI_medida'

# X = df_merged[features]
# y = df_merged[target]

# X_train, X_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_tmp, y_tmp, test_size=2/3, random_state=42)

# scaler = StandardScaler()
# X_train_s = scaler.fit_transform(X_train)
# X_val_s   = scaler.transform(X_val)
# X_test_s  = scaler.transform(X_test)

# mlp = MLPRegressor(
#     hidden_layer_sizes=(5,10),
#     activation='relu',
#     solver='adam',
#     learning_rate_init=0.001,
#     max_iter=500,
#     early_stopping=True,
#     random_state=42
# )

# mlp.fit(X_train_s, y_train)

pass



## 12. Evaluación del modelo adaptado


In [ ]:

# y_pred_test = mlp.predict(X_test_s)
# metrics(y_test.values, y_pred_test)

pass



## 13. Síntesis final

Este notebook documenta de manera completa:
- Acceso a datos WRF-SMN
- Extracción espacial y temporal
- Comparación con observaciones
- Análisis exploratorio
- Corrección mediante ML

Constituye una base sólida para una
**calibración operativa de pronósticos solares**.
